# Node2vec

Danes bomo

1. podatke predstavili kot graf,
2. izračunali vpetje vseh vozlišč,
3. uporabili vpetja vozlišč, ki pripadajo poslovnim obratom, za napovedovanje vrste obrata.

## 0. Priprava delovnega okolja

Virtualno okolje s prvih vaj iz večtabelnih podatkov (ki vključuje pandas, scikit ...), bomo razširili
z `networkx` in `node2vec`. To storite tako, da pokličete `pip install networkx node2vec`.

Če vam iz neznanega razloga nekaj sesuva globoko v nameščenih paketih, si lahko pomagate tako:

V mapo za današnje vaje (kjer ta zveščič) shranite še `mojiPaketi.txt`, ki je dostopen na učilnici.
V njej se nahajajo potrebne knjižnice in njihove verzije, ki dokazano delujejo.
Preverite, da dolge vrstice za `re3py` ni v njej (če ste datoteko prenseli med vajami, je).


Nato odprite _Ukazni poziv_ (ang. _command prompt_), ki je morda videti na začetku nekako tako

![začetni CMD](slike/zacetni_cmd.PNG)

Premaknite se v mapo za današnje vaje (ukaz `cd ...`):

![prava lokacija CMD](slike/prava_lokacija_cmd.PNG)

Naredite novo virtualno okolje: `python -m venv venv` in ga aktivirajte (`venv\Scripts\activate`).
Pred lokacijo se mora pojaviti ime okolja  (`venv`):

![aktivacija okolja](slike/aktiviran_venv.PNG)

Za vsak slučaj posodobite pip z ukazom [python -m pip install --upgrade pip](https://stackoverflow.com/questions/15221473/how-do-i-update-upgrade-pip-itself-from-inside-my-virtual-environment) 
(predvsem če imate kakšno starejšo različico pythona), nato pa z njim namestite vse potrebne pakete:

`pip install -r mojiPaketi.txt`

Čez nekaj časa bo vse nameščeno. Nato desno zgoraj (kot kaže slika) izberite pravi Python:

![izbira kernela](slike/izberi_kernel.PNG)



## 1. Pretvorba v graf

Uporabili bomo knjižnico `networkx` (ki jo naložimo z ukazom `pip install networkx`).
Pri pretvorbi v graf moramo ugotviti, kaj so vozlišča in povezave grafa.

Odgovoriti moramo na kar nekaj vprašanj, npr.

- Kako predstavimo vrstice iz tabele `business` (stolpci `business_id`, `stars`, `review_count`, `category`)?
  - Ali sploh smemo v graf vnesti informacijo o `category` (glede na dogajanje pri točki 2)?
  - Kako predstaviti vrednosti za `review_count`?
- Kako predstavimo vrstice iz tabele `attributes` (stolpci `business_id`, `name`, `value`)?
  - Tukaj je namreč pripadajoča relacija trojiška.
- Kako predstaviti vrstice iz tabele `hours` (stolpci `business_id`, `day`, `open time`, `close time`)?
   - Dve trojiški relaciji, urejenost po času

Če odgovorimo na vse to, bomo vedeli tudi, kako pretvoriti vrstice tabel `review` in `users`.

In [13]:
import os
from typing import List
import numpy as np
import pandas as pd
import networkx as nx
from node2vec import Node2Vec
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import KBinsDiscretizer


def pripona_datoteke(dim, dolzina, stevilo, p, q):
    return f"dim{dim}_dolzina{dolzina}_stevilo{stevilo}_p{p}_q{q}"

### Igranje z networkx

In [14]:
g = nx.Graph()
g.add_node("oseba Maja")
g.add_edge("oseba Maja", "mož Karel")


In [15]:
# g.nodes
g.edges

EdgeView([('oseba Maja', 'mož Karel')])

# 1. Pretvorba v graf

In [16]:
def diskretiziraj(podatki: pd.DataFrame, atributi: List[str]):
    """Diskretizira izbrane atribute, ki se nahajajo v podatkih."""
    pretvornik = KBinsDiscretizer(
        n_bins=5,
        encode="ordinal",
        strategy="quantile"
    )
    nove_vrednosti = pretvornik.fit_transform(podatki[atributi])
    for i, a in enumerate(atributi):
        podatki[a] = nove_vrednosti[:, i]

## 1. 1 Pretvorba tabele business

Večina dela je že narejenega. Popravi kodo tako, da bomo lahko vključili še kakšen numeričen atribut, ki je morda koristen.

In [17]:
def posodobi_z_business(graf: nx.Graph):
    business = pd.read_csv("yelp_business.txt", sep="\t")
    ok_atributi = [
        # nekoristno: full_address, city, state, open
        # zaenkrat neprimerno za graf: latitude, longitude, review_count
        # prevovedano: category
        "stars", "review_count"
    ]
    diskretiziraj(business, ["review_count"])
    for _, vrsta in business.iterrows():
        vozlisce_b = f"biznis_{vrsta['business_id']}"
        vozlisca_a = [f"biznis_atribut_{a}_{vrsta[a]}" for a in ok_atributi]
        povezave = [(vozlisce_b, voz_a) for voz_a in vozlisca_a]
        for od, do in povezave:
            graf.add_edge(od, do)

## 1.2 Pretvorba tabele attributes

In [18]:
def posodobi_z_attributes(graf: nx.Graph):
    """Trojiške relacije zahtevajo ustanovitev 'posebnih vozlišč'"""
    attributes = pd.read_csv("yelp_attributes.txt", sep="\t")
    # business_id	name	value
    for _, vrsta in attributes.iterrows():
        vozlisce_b = f"biznis_{vrsta['business_id']}"
        vozlisce_ime_atributa = f"atribut_{vrsta['name']}"
        vozlisce_ime_vrednost_atr = f"atribut_vrednost_{vrsta['name']}_{vrsta['value']}"
        povezave = [
            (vozlisce_b, vozlisce_ime_atributa),
            (vozlisce_ime_atributa, vozlisce_ime_vrednost_atr),
            (vozlisce_b, vozlisce_ime_vrednost_atr)
        ]
        for od, do in povezave:
            graf.add_edge(od, do)

## 1.3 Pretvorba tabele hours

In [19]:
def posodobi_z_hours(graf: nx.Graph):
    hours = pd.read_csv("yelp_hours.txt", sep="\t")
    # business_id	day	open time	close time
    for _, vrsta in hours.iterrows():
        vozlisce_b = f"biznis_{vrsta['business_id']}"
        vozlisce_d = f"dan_{vrsta['day']}"
        vozlisce_d_odpri = f"dan_odpri_{vrsta['day']}_{vrsta['open time']}"
        vozlisce_d_zapri = f"dan_zapri_{vrsta['day']}_{vrsta['close time']}"
        povezave = [
            (vozlisce_b, vozlisce_d),
            (vozlisce_b, vozlisce_d_odpri),
            (vozlisce_b, vozlisce_d_zapri),
            (vozlisce_d, vozlisce_d_odpri),
            (vozlisce_d, vozlisce_d_zapri)
        ]
        for od, do in povezave:
            graf.add_edge(od, do)

## 1.4 Pretvorba tabele review

In [20]:
def posodobi_z_review(graf: nx.Graph):
    """Povežemo biznis z recenzijo, recenzijo pa z njenimi lastnostmi"""
    reviews = pd.read_csv("yelp_review.txt", sep="\t")
    for _, vrsta in reviews.iterrows():
        # biznis <--> recenzija <--> uporabnik
        vozlisce_b = f"biznis_{vrsta['business_id']}"
        volzisce_r = f"recenzija_{vrsta['review_id']}"
        vozlisce_u = f"uporabnik_{vrsta['user_id']}"
        povezave = [
            (vozlisce_b, volzisce_r),
            (volzisce_r, vozlisce_u)
        ]
        # recenzija <--> atribut
        atributi = ["stars", "funny", "useful"]
        vozlisca_atributi = [f"recentija_atr_{a}_{vrsta[a]}" for a in atributi]
        for v_a in vozlisca_atributi:
            povezave.append((volzisce_r, v_a))
        
        for od, do in povezave:
            graf.add_edge(od, do)

In [21]:
reviews = pd.read_csv("yelp_users.txt", sep="\t")
len(reviews["fans"].unique())

37

## 1.5 Pretvoraba tabele users

In [22]:
def posodobi_z_users(graf: nx.Graph):
    """Naredi povezave uporabnik <--> atribut. Večinoma so atributi diskretizirani."""
    users = pd.read_csv("yelp_users.txt", sep="\t").fillna(-1.0)
    za_diskretizirati = [
        "review_count", "fans", "cool", "cute", "funny",
        "hot", "list", "more", "note", "photos", "plain",
        "profile", "writer", "uv_cool", "uv_funny", "uv_useful"
    ]
    # to bo povzročilo nekaj warningov ...
    diskretiziraj(users, za_diskretizirati)
    ok_atributi = ["average_rate"] + za_diskretizirati
    for _, vrsta in users.iterrows():
        vozlisce_u = f"uporabnik_{vrsta['user_id']}"
        vozlisca_u_atr = [f"u_atribut_{a}_{vrsta[a]}" for a in ok_atributi]
        for u_atr in vozlisca_u_atr:
            graf.add_edge(vozlisce_u, u_atr)


In [23]:
def yelp_to_graph():
    graf = nx.Graph()
    posodobi_z_business(graf)
    posodobi_z_attributes(graf)
    posodobi_z_review(graf)
    posodobi_z_hours(graf)
    posodobi_z_users(graf)
    return graf

## 2. Izračun vpetij

Uporabili bomo knjižnico `node2vec` (ki jo naložimo z ukazom `pip install node2vec`).
Algoritem temelji na algoritmu `word2vec`, ki je implementiran v knjižnici `gensim`,
ki se bo naložila v sklopu nameščanja knjižnice `node2vec`.

Algoritem, kot je vidno spodaj, ima kar nekaj parametrov.

In [24]:
def pridobi_vlozitev(
        graf: nx.Graph,
        dim=32,
        dolzina_sprehoda=10,
        st_sprehodov=200,
        p=2,
        q=1
):
    podrobnosti = pripona_datoteke(dim, dolzina_sprehoda, st_sprehodov, p, q)
    datoteka_model = f"node2vec_{podrobnosti}.model"
    datoteka_vektorji = f"node2vec_{podrobnosti}.csv"
    if not os.path.exists(datoteka_model):
        # verjetnosti/sprehodi
        node2vec = Node2Vec(
            graf,
            dimensions=dim,
            walk_length=dolzina_sprehoda,
            num_walks=st_sprehodov,
            p=p,
            q=q,
            workers=4
        )
        # natreniraj model
        model = node2vec.fit(window=2, min_count=1, batch_words=50)
        # shrani za kasnejšo in ponovno rabo
        model.wv.save_word2vec_format(datoteka_vektorji)
        model.save(datoteka_model)
    else:
        model = Word2Vec.load(datoteka_model)
    # preverimo, ali je vpetja/vložitev smiselno/a
    vozlisca_za_analizo = [
        'atribut_Alcohol',
        'dan_Monday'
    ]
    for v in vozlisca_za_analizo:
        print(f"Vozlišču {v} so najbolj podobna")
        for sosed, podobnost in model.wv.most_similar(v):
            print(f"    {sosed} (podobnost: {podobnost:.3f})")

# 3. Naredimo tabelo za učenje

- Iz `.csv`-ja preberemo upodobitve
- Iz tabele `business` poberemo vrednosti ciljne spremenljivke

In [25]:
def naredi_tabelo(
        dim=32,
        dolzina_sprehoda=10,
        st_sprehodov=200,
        p=2,
        q=1
):
    businesses = pd.read_csv("yelp_business.txt", sep="\t")
    biznisi = {f"biznis_{b}": i for i, b in enumerate(businesses["business_id"])}
    category = np.array(businesses["category"])
    pripona = pripona_datoteke(dim, dolzina_sprehoda, st_sprehodov, p, q)
    datoteka_vektorjev = f"node2vec_{pripona}.csv"
    with open(datoteka_vektorjev) as f:
        n_rows = len(biznisi)
        _, n_col = map(int, f.readline().strip().split(" "))
        matrix = np.zeros((n_rows, n_col))
        for row in f:
            row = row.split(" ")
            if row[0] in biznisi:
                e = [float(x) for x in row[1:]]
                matrix[biznisi[row[0]]] = np.array(e)
    return matrix, category

### Naučimo se modelov

In [26]:
yelp_graf = yelp_to_graph()
print(f"Končne statistike: {len(yelp_graf.nodes)}, {len(yelp_graf.edges)}")
pridobi_vlozitev(yelp_graf)
xs, y = naredi_tabelo()
x0, x1, y0, y1 = train_test_split(
    xs,
    y,
    test_size=0.25,
    random_state=1234,
    stratify=y
)
rf = RandomForestClassifier()  # n_estimators=300, max_features=1.0)
rf.fit(x0, y0)
y_hat = rf.predict(x1)
print("Točnost:", accuracy_score(y1, y_hat))

/home/user/programming/nsu/dn3/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/user/programming/nsu/dn3/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/user/programming/nsu/dn3/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/user/programming/nsu/dn3/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  warnings.warn(
/hom

Končne statistike: 3731, 48818


Generating walks (CPU: 4): 100%|██████████| 50/50 [00:18<00:00,  2.71it/s]


Vozlišču atribut_Alcohol so najbolj podobna
    atribut_vrednost_divey_false (podobnost: 0.996)
    atribut_vrednost_intimate_false (podobnost: 0.996)
    atribut_hipster (podobnost: 0.995)
    atribut_latenight (podobnost: 0.995)
    atribut_Has TV (podobnost: 0.995)
    atribut_lunch (podobnost: 0.995)
    atribut_Good For Groups (podobnost: 0.995)
    atribut_Waiter Service (podobnost: 0.995)
    atribut_Take-out (podobnost: 0.994)
    atribut_upscale (podobnost: 0.994)
Vozlišču dan_Monday so najbolj podobna
    dan_Wednesday (podobnost: 0.993)
    dan_Thursday (podobnost: 0.992)
    dan_Tuesday (podobnost: 0.991)
    dan_Friday (podobnost: 0.977)
    dan_Saturday (podobnost: 0.916)
    dan_Sunday (podobnost: 0.854)
    dan_zapri_Monday_13:00:00 (podobnost: 0.774)
    dan_zapri_Wednesday_13:00:00 (podobnost: 0.753)
    dan_zapri_Monday_12:30:00 (podobnost: 0.747)
    dan_odpri_Tuesday_13:30:00 (podobnost: 0.741)
Točnost: 0.8860759493670886
